In [ ]:
%run -i ../python/common.py


closeAllOpenTtySessions()
bash = BashSession()


# remember to avoid something having a history entry put a space at the beginning of the line
#bash.run(" history")

# The Shell - Part II: Some UNIX details

<!-- produced by: 
htmlFig("[{'src': '../images/UnixRunning.png'}]", 
        id="fig:runningunix", 
        align="right", 
        width="50%",
        margin="auto auto auto 1.5em",
        caption="Figure: Running Unix system.", 
        captionalign="left")
-->
<table id="fig:runningunix" align="right" width="50%" cellpadding="0" cellspacing="0" border="0" style="border-collapse: collapse; margin: auto auto auto 1.5em">
    <tr style="padding: 0; margin: 0;"> 
        <td colspan="1" width="100%" style="padding: 0; margin: 0 0 0 0; background-color:inherit;">
            <div style="padding: 0; margin: 0 0 0 0;">
              <figure style="padding: 0; margin: 0 0 0 0; width:100%;">
                   <img src="../images/UnixRunning.png" width="100%" style="padding: 0; margin: 0;">
                </figure>
            </div>
        </td>
    </tr>
    <caption align="bottom" style="text-align: left; padding: 0; margin: 0;" >
          <i>Figure: Running Unix system.</i> 
    </caption>
</table>
At this point we should have an understanding of how a running UNIX system breaks down into the Operating System (OS) kernel and user processes. The way the running software of computers system is organized and interacts is often called the "run-time" organization or structure.   Below is a brief recap of what we know so far and various associated terms, if any of this does not make sense please go back and review the prior chapters.

## Brief Recap
The OS **kernel** provides a core set of operations, **system calls**, that permit a currently running user program to work with various types of files and to start new running programs, **processes**, from **binary executable** files.  The UNIX systems provide a **terminal** orient, **shell** program that reads and executes  **command line** .  The shell can serve as a human-oriented, interactive interface, to the capabilities of the kernel.  A core feature of the shell is its ability to located and run executable files as **external commands**.   When we create new terminal windows, the UNIX OS starts a new independent shell process, whose **standard streams**, are connected to the terminal via a special **device file** type.  In this way, each new terminal gets us a new shell process that we can use to execute command lines.  

To provide additional flexibility and control the shell goes through several steps to process a command line.  This includes nine different **expansions**.  As part the command line syntax of the shell it provides us with the ability to control the Input/Output (I/O) of the command to be run, in the form of **re-directions**.  More generally, the UNIX I/O model, is one of "files" and "streams".  The kernel provides operations such as **open** that allows a process to attach an existing, or new file, as a stream.  A process can have many streams attached to it.  The kernel maintains a **file descriptor table**, for each process, the entries track streams attached to the process.  

A process uses an numeric index of file descriptor table entry to **read** bytes from or **write** bytes to a particular stream.  In turn, the bytes will come from or go to the file that the stream connects the process too.  The numeric indexes are often called **file descriptors**.  The file descriptors, **0 - standard input**, **1 - standard output**, and **2 - standard error** have special meaning as UNIX programs will by default use these as there source of data and where their normal and error data is sent.  Various re-directions operators let's us setup the streams that a command will have open before it begins to run.  

The kernel provides a special, **pipe**, file type. A Pipe allows two processes to communicate in a coordinated way.  Where one process can write to the pipe and another can read from the pipe.  The kernel takes care of the details to ensure that the two processes synchronize correctly so that no data is lost and the processes will wait as needed.  Exploiting pipes the shell provides the ability to create **pipelines** where several commands can be chained together.  The chain is formed from left to right  with the `|` symbol separating the individual simple commands.  The standard output of the left command is setup to the standard input of the right command.

Using the abilities of the kernel the shell lets us not only run external commands as  **Synchronous**, **child processes** of the shell,  but also **Asynchronously**.  By default, when we run external commands, the shell will wait for the newly started child process to finish before it goes on to read the next command line to work on.  However, using the background operator, `&`, we can tell the shell not to wait, but rather start the command as a background child process and immediately continue to the next command line.   

Various information is provided by the kernel and shell to new processes that are started. **Exported** shell variables,  "name=value" pairs,  called environment variables, are passed to newly created processes in addition to any command line arguments specified by the command line.  Copies of the open file descriptors and the **current working directory** of the new processes are inherited from the parent shell's values.   

When processes exit, they provide an **exit status** value, in the form of an integer, to the kernel.  The shell, as a parent of the child processes, it starts to run the external commands, the shell gathers the exit statuses.  The UNIX convention, is that an exit status of 0 means success and a non-zero exit status means some form of failure.  The shell lets us examine and use the exit status of external command to create various programming structures such as conditionals, if statements and loops.

Finally, in addition to supporting interactive use, where the shell waits for the next command line to be entered by us at the terminal, the shell program  can be started with an argument that is a **shell script** file from which it will read the command lines to execute.  

## Overview

In this chapters we will round out our understanding of the core concepts that the running software of a computer system is built around.  Continuing to use UNIX as our example we will introduce the idea of users, privilege, permissions and credentials. We will then revisit Files and Streams along with Processes.  As we go along we will both introduce UNIX shell commands that let us explore the concepts and we will expand our view of what is going on in the kernel.  Finally we conclude by putting all the pieces together to provide a more detailed view of a running UNIX system. 

## Multi-User Operating Systems

One of the defining features of UNIX is its origin as an operating system that allows multiple people to "securely" share, and use, a single computer, at the same time.  UNIX's popularity as the operating system used on the server computers of the internet have put its model and mechanisms for sharing a computer securely, at center of modern computer security.  Much of what we hear about on the news and see in movies when it comes to security will often involve UNIX systems in one way or another. 

Our goals in this section is to introduce the core terms and ideas that extend the notion of having multiple running programs, to having them potentially associated with different users.

### Multi-programming/tasking

Given what we are familiar with on most of our "devices", we might take it for granted, that computer run-times support [several running programs at the same time](../unix/shellintro.ipynb#sec:processcreation).  However, this is not the only run-time organization for computing systems.  In the past, operating systems for personal computers, such as [DOS](https://en.wikipedia.org/wiki/DOS) focused on running a single program/task at a time.  The software that forms the operating system "kernel", for such single-tasking run-times, are generally much simpler, and require fewer hardware features.  To this end when their is no need for multiple processes, as is the case today, on the simple computers that are embedded in things like smart light bulbs or buttons, we continue to use single-tasking run times.  

```{note}
An example, of a single tasking runtime, that you might be familiar with, is the [Arduino](https://en.wikipedia.org/wiki/Arduino) run-time.  The original Arduino boards such as the [UNO](https://en.wikipedia.org/wiki/Arduino_Uno) are very simple with respect to the way the run-time is organized. A single application, program, is written, compiled and uploaded to the board.  The "bootloader", such as [optiboot](https://github.com/Optiboot/optiboot) along with the libraries of software that come with the [Arduino software development environment](https://en.wikipedia.org/wiki/Arduino#IDE) act as "library OS".   Where, where the bootloader enables the application to be uploaded on to the device and start the application running.  The included libraries provide a set of calls that make it easier to make use of the hardware features of the board such as controlling and interacting with buttons, lights, motors, displays etc.  
```

#### Multi-Processing

Run-times that are designed to run more than one program at a time are generally referred to as multi-processing or multi-tasking systems.  Typically, on such systems several running programs can be started at the same time. Beyond the basic ability to start several, concurrently running programs, systems can vary in a great deal as to the level of transparency and independence the running programs have with respect to each other and the operating system software.  

Clearly, UNIX is a multi-processing system.  UNIX is built around the notion that an individual process should largely be able to ignore that it is sharing the computer with other processes.  As a matter of fact, a program written to run on UNIX can be developed as if it was running on the computer alone, along with the kernel.  It is the UNIX kernel's job to ensure that this illusion of isolated use and independence is maintained.  

Operating systems such as UNIX work very hard to try and ensure that processes cannot interfere with each other.  By default processes are independent and cannot look "inside" another process or modify the behavior of another process.  This includes the kernel.  A process, cannot look inside the kernel or modify the kernels behavior.    This ensures that even if one program is buggy the other processes and the kernel will not be affected by the buggy program.  As a simple example, consider a program that accidentally has an infinite loop -- loops forever not doing anything.  Running such a program should not make the computer unresponsive or stop other running programs from executing correctly.  The operating system's kernel, must be carefully written and must make use of various hardware features, to enact this level of isolation.  The goal is to ensure that the kernel is written to ensure that when running the kernel and processes are protected from the actions of any other process.  

### Multi-user

In addition to running multiple programs at the same time, operating systems can be designed to share a computer across several different users.  In the early days of computing this was the normal model.  Computers were large expensive devices and inherently needed to be shared to be cost effective.  The UNIX design is deeply influenced by its history as an operating system for computers shared by many independent users.  

To understand this let's take a moment to consider the opposite model -- personal computing devices such as early Personal Computers (PC's) and today's Smart phones.  In these cases, each user, owns their own device.  All the software and data on the computer belongs to the physical owner of the device.  In such cases, the operating systems need not support the idea of the device being used by several users and thus does not need to protect or hide information.  There is one owner, who is also the only user of the device, and has complete control.  On such systems, protecting running processes and the kernel is still a useful property.  It helps avoid bugs or mistakes from crashing the system to the point where it is not usable or information is lost.   However, more complex ideas like independent user accounts and files are not a necessity.  

In contrast, UNIX was designed from the beginning to support a "multi-user" environment.  So in addition to supporting the execution of multiple processes the system also needs to ensure that the processes, software and data of one user can be isolated, protected and even hidden from other users.    In other words, the owners of a computer and the users of the computer need not be the same people nor should they have the same "rights" to do certain things.  To this end UNIX developed various ideas for allowing multiple "user accounts".  Where the system supports the ability to isolate the processes and information across user accounts.   Additionally, UNIX has a flexible model that also allows users to exploit the fact that they are on a shared system and selectively cooperate by sharing processes and information if they want to do so. 

Regardless of supporting flexibility to cooperate and share information across users, UNIX must always ensure that the kernel is protected from all user processes and can never be, either intentionally or accidentally, compromised.  After all, given the that the kernel is responsible for ensuring that processes are protected from each other if it can be subverted by an arbitrary user that user can then do anything they want with respect to the file and processes of other users.

As we will see in UNIX there is a special user, the `root` user, that in some sense represents the owner of the system and has the rights to create, modify and remove users accounts.  The `root` user even has sufficient rights to comprise the kernel and as such access to this user account must be carefully protected.   This might seem odd at first to have such a powerful user, but such a user is required to install the operating system and updated it as needed.  

```{note}
So while UNIX started its life as an operating system for computers shared among several users, today it is also used on personal computing devices, such as laptops, tablets, smartphones, e-readers, watches, etc.  It should not be hard to see that a system designed to support several users can easily be used by a single user by simply not creating multiple user accounts.  Further using an operating system that has support for multiple users on a device used by a single user has the several advantages.  One by creating an account for your daily work that does not have rights to modify all the files and information of the system can avoid one from making catastrophic mistakes that could result in crashing the system and looking data.  Further if you do decide to share your devices with someone you can easily create a separate account for them and they will not have access to your personal data. 
```

```{note}
The associated Jupyter software environment of this book runs in what is called a Linux container.  It provides a reader with access to what is effectively a private single user Linux UNIX computer.  Once you login into the JupyterHub service and you start your "server" the cloud service starts an independent UNIX user environment, a container, for you with on top of a UNIX kernel.   This is an isolated and independent UNIX user environment that is for your use.  There is a single main user who's name is `jovyan`.   Upon startup the software is configured to start a  program called the "Jupyter Lab" server on your behalf.  Once this software is running your web browser is directed to communicate with your Jupyter Lab server.   This software sends to your browser the code (javascript) to allow you start terminal sessions.  In this environment your identity is the `jovyan` user.  The shells, `bash` processes, that are started for each terminal you create are automatically login as the `jovyan` user.  Think of this as a your personal UNIX system and we encourage you to poke around and play!
```

#### Split, two-level, security model

So while many independent processes can be concurrently running a more fundamental separation of the runtime exits -- that between the kernel and all "user" processes. 

##### Privileged (Kernel)

As we have seen, the kernel is an ever present part of the runtime structure of a UNIX system.  When the computer is booted the kernel is started and at least one user process is launched.  After this many other user process can get started and be terminated.  No matter how many or how few user processes exits the kernel will always be present.  It is constantly taking care of many details of keeping the computer running.  It is constantly monitoring and responding to the activity on the I/O devices such as terminals, networks, directly attached keyboards, etc.  Further more, as we have seen it is fielding requests for its system calls from the currently running processes.  For that matter, it is also ensuring that each currently running processes is getting its chance to execute.  The kernel makes use of various special features of the hardware to both ensure that the processes stay isolated and to directly access the various devices of the computer.  

To enable this separation and control the kernel must run in a special mode of the hardware that gives it access to all the advanced features it needs.  We think of the kernel running in a special hardware enabled "privileged" mode.  Thus we think of the kernel as the "privileged" software where it has access to the capabilities of the hardware that processes do not.  In the next Part of this book, when we delve into what exactly the software of a program is and how it interacts with the hardware, at that point we will be able to more precisely understand what "privileged" mode is.  

```{note}
While the UNIX runtime model is designed around a simple two level model of hardware enforced privilege the hardware itself might offer richer models that support several modes or "ring" of privilege/protection.  Despite this the UNIX model only requires two and its simple model has ensured that it can be widely supported on various hardware systems.
```

##### Non-privileged (User Processes)

In contrast to the privileged mode that kernel software runs in all processes run in a non-privileged mode.  In this mode if they attempt to directly do a privileged operation such as directly accessing the hardware that enforces privilege the hardware will stop the process and hand control over to the OS kernel.  As part of this the hardware will switch into privileged mode and let the kernel decide what to do.  

In this way all processes are considered to be operating in the same non-privileged mode which we simply refer to as user mode.  So we refer to all processes as user mode execution.   On top of this simple two level model, kernel and user, the kernel must build up a mechanisms for distinguishing the processes and files with respect to individual users.

Given that all processes are started by the kernel and that all processes make calls to the kernel to make use of files the kernel can create distinction between the processes as belong to particular users by maintaining internal information.  Using this information the kernel can permit or deny access by allow a call made by a process to succeed or fail.  

At a high level the distinction between users, is as simple as writing the kernel software to conduct tests on every system call.  The tests confirms if the user of the process making the request has the permissions to do the requested operation.  If it does the operation is allowed to proceed and if not the operation is denied and the system call returns a failure to the calling process.  Much of what follows is adding details on how this high level idea is translated in practice. 

```{note}
In the diagrams we use to visualize a running system when we place items within the boundary of the kernel it is important to remember this interior content is not visible or accessible to the processes.  The only way for a process to access something outside of itself is to make system calls (later we will extend this to include "exceptions") to the kernel.  The kernel then decides how to handle any given request from the particular process making the request.  With this in mind we see that it is really the software of the OS kernel that create the view of the computer for a process.  To help us understand the standard view the UNIX kernel presents to processes for interacting with the resources of the computers we will draw objects within the kernel boundaries.
```


### UNIX users and groups

In order to permit the distinction between users of the computer the UNIX kernel maintains a table of valid users.  Each valid **user** is assigned a unique integer identifier called a **user id (uid)**.  To implement the notion of ownership and permissions we will see that resources will be stamped with uid's to indicate what user owns a particular resource.  In addition to the numeric uid UNIX keeps track of several pieces of information for each uid including a short ASCII human readable users name.

#### `whoami` and `id`

Let's run a few shell commands that display information about our current user.

In [ ]:
bash.run("whoami")

The `whoami` command prints the human friendly user name of the process launched to run the command.  As we can see our user name is `jovyan`. The `id` command can be used to print various information regarding the id's of a process.  Passing in the `-u` option tells `id` to only print the uid of the process.

In [ ]:
bash.run("id -u")

`id -u` prints the numeric uid of our user.  

To provide greater flexibility UNIX also support the notion of groups


#### whoami and id

#### /etc/passwd and /etc/group

#### The `root` user

### Kernel View

- table of authorized users and groups

#### Traditional: User Group Credential Model

#### Modern: Capabilities

## Revisiting UNIX Files and Streams

### Types of Files

### Meta data

#### Ownership and Permissions

#### Operations and Permissions

### Kernel View

- file as objects in kernel 
- regular and special 


## Revisiting UNIX Processes

### Process Management Commands

#### Background Commands

##### Jobs and Processes

##### Signals

### Kernel View


### Putting it all together

#### Pipes and Pipelines
